In [120]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [121]:
#load model, scalar pickle, onehot
model = load_model('churn_model.h5')

#load encoder and sclar
with open('onehot_encoder_geography.pkl', 'rb') as file:
    encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as genfile:
    encoder_gender = pickle.load(genfile)

with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [122]:
#example input data
ip_dt = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender' : 'Female',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [123]:
geo_encoded = encoder_geo.transform([[ip_dt['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\hetpa\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [124]:
inp_df = pd.DataFrame([ip_dt])
inp_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,40,3,60000,2,1,1,50000


In [125]:
inp_df['Gender'] = encoder_gender.transform(inp_df['Gender'])

In [126]:
inp_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [127]:
#concatenate OHE
inp_df = pd.concat([inp_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [128]:
inp_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [116]:
#scale inp data
scaled_inp = scalar.transform(inp_df)

In [117]:
scaled_inp

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [118]:
#prediction churn
pred = model.predict(scaled_inp)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


array([[0.09845037]], dtype=float32)

In [119]:
if pred > 0.5:
    print("Customer will churn")
else:
    print("Customer will not churn")

Customer will not churn
